In [11]:
import pandas as pd
import numpy as np
import time
import datetime
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

# Utils

In [12]:
def split_symbol(symbol):
    return symbol.split('.')[0]

In [13]:
def mapper(symb, mapper):
    try:
        return mapper.get(symb)
    except:
        return 'Empty'

In [14]:
def infer_rating(df, qnt_col = 'SHARESQTY', price_col= 'SHAREPRICE', rating_col = 'RATING'):

    order_prices = df[qnt_col] * df[price_col]
    scaler = MinMaxScaler(feature_range=(1,5))
    scaled_price = scaler.fit_transform(order_prices.values.reshape(-1, 1))
    df[rating_col] = np.clip(scaled_price, 1, 5)
    return df

In [15]:
def get_max_values(df, col_name = 'RATING', round_method = 'round'):
    max_rating_row = df.loc[df[col_name].idxmax()]
    if round_method == 'round':
        max_rating_row[col_name] = max_rating_row[col_name].round(0)
        
    elif round_method == 'ceil':
        max_rating_row[col_name] = np.ceil(max_rating_row[col_name].array)
    
    return max_rating_row

In [16]:
def to_timestamp(date):
    return datetime.datetime.timestamp(date)

# Preprocessing

In [17]:
portfolios = pd.read_excel(
    "../../data/PORTFOLIODETAILS_0403.xlsx",
    sheet_name = ['Sheet 1','Sheet 2'])
# portfolios.shape

In [18]:
stock_info = pd.read_excel('../../data/stock_data.xlsx')
stock_info = stock_info.drop(['Unnamed: 0'],axis = 1)
stock_info.shape

(282, 4)

In [19]:
stock_info = stock_info.dropna()

In [20]:
stock_info.head()

,symbol,name,buisnesssummary,gics_code
0,HBS,hSenid Business Solutions PLC,An indigenous multinational catering towards m...,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Manufacturing tyres and tubes and marketing lo...,Automobiles & Components
2,ABL,AMANA BANK PLC,unknown,Banks
3,DFCC,DFCC BANK PLC,The principal activities of DFCC Bank include ...,Banks
4,COMB,COMMERCIAL BANK OF CEYLON PLC,Commercial Banking,Banks


In [21]:
unique_symbols = set(stock_info.symbol.unique())
len(unique_symbols)

280

In [22]:
portfolios_df = pd.concat([portfolios['Sheet 1'],portfolios['Sheet 2']], ignore_index= True)
portfolios_df = portfolios_df.loc[portfolios_df.TRAN_TYPE == 'B']

In [23]:
portfolios_df.shape

(38215, 8)

In [24]:
portfolios_df.head()

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME
0,RPS-40657-LI/00,LOFC.N0000,2022004016,B,200,29.1,2022-01-10,1/10/2022 10:55:08.000000 AM
1,RPS-696600287-VN/00,LALU.N0000,2022004017,B,1000,37.4,2022-01-10,1/10/2022 10:55:09.000000 AM
5,RPS-40657-LI/00,LOFC.N0000,2022004021,B,1801,29.3,2022-01-10,1/10/2022 10:56:53.000000 AM
6,RPS-732590072-VN/00,LALU.N0000,2022004022,B,5000,38.0,2022-01-10,1/10/2022 10:56:57.000000 AM
9,RPS-743172922-VN/00,EXPO.N0000,2022004025,B,21,394.0,2022-01-10,1/10/2022 11:38:11.000000 AM


In [25]:
portfolios_df_fil_1 = portfolios_df.groupby(by = 'CDSACCNO').filter(lambda x: x['STOCKCODE'].nunique() > 10)
portfolios_df_fil_1['UNIX_TS'] = portfolios_df_fil_1['TRADE_DATE'].apply(lambda x: to_timestamp(x))
portfolios_df_fil_1.head()

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME,UNIX_TS
0,RPS-40657-LI/00,LOFC.N0000,2022004016,B,200,29.1,2022-01-10,1/10/2022 10:55:08.000000 AM,1.641753e+09
1,RPS-696600287-VN/00,LALU.N0000,2022004017,B,1000,37.4,2022-01-10,1/10/2022 10:55:09.000000 AM,1.641753e+09
5,RPS-40657-LI/00,LOFC.N0000,2022004021,B,1801,29.3,2022-01-10,1/10/2022 10:56:53.000000 AM,1.641753e+09
6,RPS-732590072-VN/00,LALU.N0000,2022004022,B,5000,38.0,2022-01-10,1/10/2022 10:56:57.000000 AM,1.641753e+09
9,RPS-743172922-VN/00,EXPO.N0000,2022004025,B,21,394.0,2022-01-10,1/10/2022 11:38:11.000000 AM,1.641753e+09


In [26]:
portfolios_df_fil_1.CDSACCNO.nunique(), portfolios_df_fil_1.STOCKCODE.nunique()

(139, 306)

In [27]:
prev_symbols = set(list(portfolios_df_fil_1.STOCKCODE.unique()))
portfolios_df_fil_1.shape

(30457, 9)

In [28]:
portfolios_df_fil_1['STOCKCODE'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x : split_symbol(x))

In [29]:
portfolios_df_fil_1['STOCKCODE'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x : split_symbol(x))

In [30]:
unique_port_symbols = set(portfolios_df_fil_1.STOCKCODE.unique())
len(unique_port_symbols)

278

In [31]:
to_remove = list(unique_port_symbols - unique_symbols)
to_remove

['AGPL', 'CBNK', 'WIND', 'CITW', 'UBF', 'LGIL', 'YORK', 'SFL', 'CLC', 'WATA']

In [32]:
portfolios_df_fil_1 = portfolios_df_fil_1[~portfolios_df_fil_1.STOCKCODE.isin(to_remove)]

In [33]:
symb_to_name = dict(zip(stock_info.symbol,stock_info.name))
symb_to_gics = dict(zip(stock_info.symbol, stock_info.gics_code))

In [34]:
portfolios_df_fil_1['STOCKNAME'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x: mapper(x,symb_to_name))
portfolios_df_fil_1['GICS'] = portfolios_df_fil_1.STOCKCODE.apply(lambda x: mapper(x,symb_to_gics))

In [35]:
# portfolios_df_fil_1[portfolios_df_fil_1['STOCKNAME'] == np.nan]

In [36]:
portfolios_df_fil_1.CDSACCNO.nunique(), portfolios_df_fil_1.STOCKCODE.nunique()

(139, 268)

In [37]:
# portfolios_df_fil_3 = filter_portfolios(portfolios_df_fil_2)
portfolios_df_fil_3 = portfolios_df_fil_1.groupby('CDSACCNO', group_keys= False).apply(lambda x: infer_rating(x)).groupby(['CDSACCNO','STOCKCODE'], group_keys= False).apply(lambda x: get_max_values(x)).reset_index(drop =True).sort_values('RATING', ascending= False)

C:\Users\naradaw\AppData\Local\Temp\ipykernel_18524\1404588810.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  portfolios_df_fil_3 = portfolios_df_fil_1.groupby('CDSACCNO', group_keys= False).apply(lambda x: infer_rating(x)).groupby(['CDSACCNO','STOCKCODE'], group_keys= False).apply(lambda x: get_max_values(x)).reset_index(drop =True).sort_values('RATING', ascending= False)
C:\Users\naradaw\AppData\Local\Temp\ipykernel_18524\1404588810.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupin

In [38]:
portfolios_df_fil_3.CDSACCNO.nunique(), portfolios_df_fil_3.STOCKCODE.nunique()

(139, 268)

In [39]:
portfolios_df_fil_3.head(2)

,CDSACCNO,STOCKCODE,REFERANCE,TRAN_TYPE,SHARESQTY,SHAREPRICE,TRADE_DATE,TRADE_TIME,UNIX_TS,STOCKNAME,GICS,RATING
1587,RPS-583233075-VN/00,VLL,2022031882,B,14000,7.1,2022-08-10,8/10/2022 11:02:10.000000 AM,1.660070e+09,VIDULLANKA PLC,Utilities,5.0
3429,RPS-883361873-VN/00,NDB,2022019743,B,250,63.5,2022-03-03,3/3/2022 12:22:16.000000 PM,1.646246e+09,NATIONAL DEVELOPMENT BANK PLC,Banks,5.0


In [40]:
portfolios_df_fil_3.GICS.nunique()

33

In [41]:
portfolios_df_fil_4 = portfolios_df_fil_3[['CDSACCNO','STOCKCODE','UNIX_TS','RATING','GICS','STOCKNAME']] #,'GICS','STOCKNAME'

In [42]:
data_dict = portfolios_df_fil_4.to_dict(orient='list')

In [43]:
dataset = tf.data.Dataset.from_tensor_slices(data_dict)

In [44]:
next(iter(dataset.batch(1)))

{'CDSACCNO': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'RPS-583233075-VN/00'], dtype=object)>,
 'STOCKCODE': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'VLL'], dtype=object)>,
 'UNIX_TS': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.6600698e+09], dtype=float32)>,
 'RATING': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>,
 'GICS': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Utilities'], dtype=object)>,
 'STOCKNAME': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'VIDULLANKA PLC'], dtype=object)>}

In [45]:
dataset.save("../../data/portfolios_tfds_lists")

In [46]:
len(dataset)

3847

In [47]:
tf.random.set_seed(42)
shuffled = dataset.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(len(dataset)* 0.8))
test = shuffled.skip(int(len(dataset)* 0.8)).take(int(len(dataset)* 0.2))

In [48]:
train.save("../../data/train_lists")
test.save("../../data/test_lists")

In [46]:
new_dataset = tf.data.Dataset.load("../../data/atrad_portfolios/portfolios_tfds")

# Work Here